### Prévision taux de régularité des TER de la SNCF

In [4]:
# !pip install statsmodels

In [5]:
#filtration des warnings
import warnings
warnings.filterwarnings("ignore")

In [6]:
# afficher tout le contenu des cellules
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Nous utilisons essentiellement le module `tsa` de `statsmodels` qui regorge essentiellement les fonctions et objets pour modéliser les séries temporelles.

In [7]:
import statsmodels as stm
import statsmodels.tsa as tsa
import requests as rq
import matplotlib.pyplot as plt
import pandas as pd

#### Récupération et examen des données

In [8]:
# Récupération des données sur la régularité des TER (regularite-mensuelle-ter)
url_api_reg_ter = "https://ressources.data.sncf.com/api/v2/catalog/datasets/regularite-mensuelle-ter/exports/json?limit=-1&offset=0&lang=fr&timezone=Europe%2FParis"
regTER = rq.get(url_api_reg_ter)
regTER = regTER.json() 
regTER = pd.json_normalize(regTER)


In [9]:
regTER.shape
regTER.head()
regTER.tail()

(1825, 9)

,date,region,nombre_de_trains_programmes,nombre_de_trains_ayant_circule,nombre_de_trains_annules,nombre_de_trains_en_retard_a_l_arrivee,taux_de_regularite,nombre_de_trains_a_l_heure_pour_un_train_en_retard_a_l_arrivee,commentaires
0,2013-01,Languedoc Roussillon,5024.0,4897.0,127.0,377.0,92.301409,11.989390,Chute de neige sur le littoral.
1,2013-01,Midi Pyrénées,8208.0,7941.0,267.0,903.0,88.628636,7.794020,None
2,2013-01,Provence Alpes Côte d'Azur,13956.0,13219.0,737.0,1816.0,86.262198,6.279185,None
3,2013-01,Rhône Alpes,31438.0,30779.0,659.0,3552.0,88.459664,7.665259,None
4,2013-02,Alsace,18263.0,18097.0,166.0,771.0,95.739625,22.472114,Intempéries.


,date,region,nombre_de_trains_programmes,nombre_de_trains_ayant_circule,nombre_de_trains_annules,nombre_de_trains_en_retard_a_l_arrivee,taux_de_regularite,nombre_de_trains_a_l_heure_pour_un_train_en_retard_a_l_arrivee,commentaires
1820,2022-08,Auvergne-Rhône-Alpes,36375.0,35712.0,663.0,2799.0,92.162298,11.758842,None
1821,2022-08,Hauts-de-France,25408.0,24778.0,630.0,2171.0,91.238195,10.413174,None
1822,2022-10,Occitanie,13081.0,12618.0,463.0,1329.0,89.467427,8.494357,None
1823,2022-10,Provence Alpes Côte d'Azur,13798.0,13403.0,395.0,1383.0,89.681415,8.691251,None
1824,2022-11,Grand Est,38434.0,37362.0,1072.0,2934.0,92.147101,11.734151,None


In [10]:
regTER.columns
regTER.describe()

Index(['date', 'region', 'nombre_de_trains_programmes',
       'nombre_de_trains_ayant_circule', 'nombre_de_trains_annules',
       'nombre_de_trains_en_retard_a_l_arrivee', 'taux_de_regularite',
       'nombre_de_trains_a_l_heure_pour_un_train_en_retard_a_l_arrivee',
       'commentaires'],
      dtype='object')

,nombre_de_trains_programmes,nombre_de_trains_ayant_circule,nombre_de_trains_annules,nombre_de_trains_en_retard_a_l_arrivee,taux_de_regularite,nombre_de_trains_a_l_heure_pour_un_train_en_retard_a_l_arrivee
count,1788.000000,1788.000000,1788.000000,1788.000000,1788.000000,1788.000000
mean,12292.477629,12039.897651,258.276846,1011.242729,91.574464,13.044802
std,9015.810259,8818.687906,291.916890,837.461109,3.420634,6.218286
min,1129.000000,1125.000000,4.000000,35.000000,74.365711,2.901025
25%,5901.500000,5825.750000,83.000000,421.000000,89.670614,8.681130
50%,9474.000000,9263.500000,157.000000,729.500000,92.017786,11.527859
75%,14888.000000,14406.500000,333.000000,1239.500000,94.124250,16.019116
max,46329.000000,45569.000000,4024.000000,4576.000000,98.026494,49.671233


`taux_de_regularite` est la variable cible du modèle à mettre en oeuvre

#### Comment est calculé le taux de régularité ?

#### Traitement des valeurs manquantes

In [11]:
regTER[regTER["taux_de_regularite"].isnull()]
regTER[regTER["taux_de_regularite"].isnull()].shape

,date,region,nombre_de_trains_programmes,nombre_de_trains_ayant_circule,nombre_de_trains_annules,nombre_de_trains_en_retard_a_l_arrivee,taux_de_regularite,nombre_de_trains_a_l_heure_pour_un_train_en_retard_a_l_arrivee,commentaires
9,2013-02,Lorraine,NaN,NaN,NaN,NaN,NaN,NaN,Le Président de la Région Lorraine s'est oppos...
41,2013-12,Lorraine,NaN,NaN,NaN,NaN,NaN,NaN,Le Président de la Région Lorraine s'est oppos...
58,2014-03,Lorraine,NaN,NaN,NaN,NaN,NaN,NaN,Le Président de la Région Lorraine s'est oppos...
71,2013-03,Lorraine,NaN,NaN,NaN,NaN,NaN,NaN,Le Président de la Région Lorraine s'est oppos...
81,2013-04,Lorraine,NaN,NaN,NaN,NaN,NaN,NaN,Le Président de la Région Lorraine s'est oppos...
84,2014-08,Lorraine,NaN,NaN,NaN,NaN,NaN,NaN,Le Président de la Région Lorraine s'est oppos...
102,2014-10,Lorraine,NaN,NaN,NaN,NaN,NaN,NaN,Le Président de la Région Lorraine s'est oppos...
156,2015-05,Lorraine,NaN,NaN,NaN,NaN,NaN,NaN,Le Président de la Région Lorraine s'est oppos...
169,2014-01,Lorraine,NaN,NaN,NaN,NaN,NaN,NaN,Le Président de la Région Lorraine s'est oppos...
180,2015-08,Lorraine,NaN,NaN,NaN,NaN,NaN,NaN,Le Président de la Région Lorraine s'est oppos...


(37, 9)

In [19]:
regTER["region"].unique()
regTER["region"].unique()

array(['Languedoc Roussillon', 'Midi Pyrénées',
       "Provence Alpes Côte d'Azur", 'Rhône Alpes', 'Alsace',
       'Franche Comté', 'Haute Normandie', 'Lorraine', 'Auvergne',
       'Centre', 'Champagne Ardenne', 'Basse Normandie', 'Limousin',
       'Nord Pas de Calais', 'Poitou Charentes', 'Pays-de-la-Loire',
       'Picardie', 'Aquitaine', 'Bretagne', 'Bourgogne', 'Grand Est',
       'Auvergne-Rhône-Alpes', 'Normandie', 'Bourgogne-Franche-Comté',
       'Hauts-de-France', 'Occitanie', 'Nouvelle Aquitaine',
       'Centre Val-de-Loire'], dtype=object)

28 régions de la SNCF sont présentes dans les données. 

Nous utilisons la méthode des deux plus proches voisins pour inputer 